## Connect to Snowflake

In [3]:
import snowflake.connector
from snowflake_credentials import snowflake_params

conn = snowflake.connector.connect(
    user=snowflake_params['user'],
    password=snowflake_params['password'],
    account=snowflake_params['account'],
    warehouse=snowflake_params['warehouse'],
    database=snowflake_params['database'],
    schema=snowflake_params['schema'],
    role=snowflake_params['role']
)

cur = conn.cursor()

## Preview the Table Data

In [21]:
businesses_count = cur.execute('''
                               SELECT COUNT(*)
                               FROM yelp_businesses
                               ''').fetch_pandas_all()

print(f'There are {businesses_count['COUNT(*)'][0]:,} records in the Yelp businesses table.')

df = cur.execute('''
                 SELECT *
                 FROM yelp_businesses 
                 LIMIT 5
                 ''').fetch_pandas_all()

display(df)

There are 150,346 records in the Yelp businesses table.


,NAME,BUSINESS_ID,IS_OPEN,REVIEW_COUNT,STARS,CATEGORIES,STATE,CITY,POSTAL_CODE,LATITUDE,LONGITUDE
0,"Abby Rappoport, LAC, CMQ",Pns2l4eNsfO8kk83dixA6A,0,7,5,"Doctors, Traditional Chinese Medicine, Naturop...",CA,Santa Barbara,93101,34.426679,-119.711197
1,The UPS Store,mpf3x-BjTdTEA3yCZrAYPw,1,15,3,"Shipping Centers, Local Services, Notaries, Ma...",MO,Affton,63123,38.551126,-90.335695
2,Target,tUFrWirKiKi_TAnsVWINQQ,0,22,4,"Department Stores, Shopping, Fashion, Home & G...",AZ,Tucson,85711,32.223236,-110.880452
3,St Honore Pastries,MTSW4McQd7CbVtyjqoe9mw,1,80,4,"Restaurants, Food, Bubble Tea, Coffee & Tea, B...",PA,Philadelphia,19107,39.955505,-75.155564
4,Perkiomen Valley Brewery,mWMc6_wTdE0EUBKIGXDVfA,1,13,5,"Brewpubs, Breweries, Food",PA,Green Lane,18054,40.338183,-75.471659


In [22]:
reviews_count = cur.execute('''
                               SELECT COUNT(*)
                               FROM yelp_reviews
                               ''').fetch_pandas_all()

print(f'There are {reviews_count['COUNT(*)'][0]:,} records in the Yelp reviews table.')

df = cur.execute('''
                 SELECT *
                 FROM yelp_reviews 
                 LIMIT 5
                 ''').fetch_pandas_all()

display(df)

There are 6,990,280 records in the Yelp reviews table.


,BUSINESS_ID,REVIEW_ID,USER_ID,REVIEW_DATE,USER_REVIEW,STARS,COOL_VOTES,FUNNY_VOTES,USEFUL_VOTES
0,iRIHK8-EwpeffwvoO4nzIA,oXpWjOG2rue-2bHbjvDZIQ,1uHAAjOX18Px4OibCMI3pg,2018-10-04,Awesome spot. This is one of my favorite place...,5,0,0,0
1,U9TJ13_B6mySAb1RI4lhug,QVL6hH7M0nv5ljHCjRfK0w,KZXmUq6y_0k4guwAfPmQcQ,2017-05-13,"I like Rumor, but some nights, Idk why they ch...",3,0,0,0
2,oBNrLz4EDhiscSlbOl8uAw,PAjnd_I5pkN2m9Mg5cJTUg,C8jSfs5zj3NkqBx-YRb3tA,2018-04-15,This place has the rudest server I think I hav...,1,0,0,1
3,IL7MEBxfei9pOaGR1g6joQ,H_gK3gJrBSMlxELs2BC7sQ,YOIorhfo8hHhilRyam2c4w,2017-12-29,I've had the ice cream from here a number of t...,4,0,0,0
4,t-cT3vDv6vB0AAow_XIK8g,kViZwGpixoAnpN7xpsaJyQ,y6RQ8oBEtryoXVTxBddNng,2012-06-26,This is a very nice store to visit. They have ...,5,0,0,0


In [23]:
users_count = cur.execute('''
                               SELECT COUNT(*)
                               FROM yelp_users
                               ''').fetch_pandas_all()

print(f'There are {users_count['COUNT(*)'][0]:,} records in the Yelp users table.')

df = cur.execute('''
                 SELECT *
                 FROM yelp_users 
                 LIMIT 5
                 ''').fetch_pandas_all()

display(df)

There are 1,987,897 records in the Yelp users table.


,FIRST_NAME,USER_ID,YELPING_SINCE,REVIEW_COUNT,AVG_STARS,FANS,COOL_COMPLIMENTS,CUTE_COMPLIMENTS,FUNNY_COMPLIMENTS,HOT_COMPLIMENTS,...,MORE_COMPLIMENTS,NOTE_COMPLIMENTS,PHOTO_COMPLIMENTS,PLAIN_COMPLIMENTS,PROFILE_COMPLIMENTS,WRITER_COMPLIMENTS,COOL_VOTES,ELITE,FUNNY_VOTES,USEFUL_VOTES
0,David,VPQ3ZKv1BY87UWuERLqoYQ,2010-07-05,13,4.31,0,0,0,0,0,...,0,0,0,0,0,0,11,,5,46
1,Pomeloh,Blu3JBisecCoFuwzV3A3eA,2012-03-29,14,4.40,0,0,0,0,0,...,0,0,0,0,0,0,7,,2,12
2,Alana,lDbh9HBWxyEM_ilY5ER_dg,2011-10-31,6,5.00,0,0,0,0,0,...,0,0,0,1,0,0,4,,0,6
3,Nancy,lkTKWP8Y-BN_8ggxrls2rA,2012-01-16,27,4.00,0,0,0,0,0,...,0,1,0,0,0,0,3,,1,17
4,Maureen,o57OsxkvddOIMpj_qiprSw,2011-07-08,3,3.33,0,0,0,0,0,...,0,1,0,0,0,0,0,,0,2


In [24]:
tips_count = cur.execute('''
                               SELECT COUNT(*)
                               FROM yelp_tips
                               ''').fetch_pandas_all()

print(f'There are {tips_count['COUNT(*)'][0]:,} records in the Yelp tips table.')

df = cur.execute('''
                 SELECT *
                 FROM yelp_tips 
                 LIMIT 5
                 ''').fetch_pandas_all()

display(df)

There are 908,915 records in the Yelp tips table.


,BUSINESS_ID,USER_ID,DATE,TIP,COMPLIMENT_COUNT
0,WnHG_obpmHazbngTdu570Q,_Wkf2RNDnBuqMOpzSqmZXw,2015-08-14,Only a couple more weeks,0
1,c-CAcdH2-12g1sQUQfy5xw,xnuEEuMfvCNLFMtaZf7asw,2016-03-20,Get here early!!,0
2,wj8XtPyuREj8_0GQz3LZ6w,W0DJOPsSwcAj0uqCJG8iLw,2015-08-14,If you have not tried this place what are you ...,0
3,l8vwslfqKq1yxlmgVi-M0Q,AED6tP0v4aMPd9-YtjPcPg,2015-05-11,Check out the third floor lounge for nice city...,0
4,KZe5kpIHWVuJl6WgNYzceg,9UZlB3fHVe2JBvPw4kRb5g,2012-03-03,Pancakes with Breakfast Trick?,0


## Category Analysis

#### Number of Businesses by Category

In [ ]:
# Top 20 categories of all businesses on Yelp.
df = cur.execute('''
                 WITH cte AS (
                    -- split the categories column so each business has a row for each category it has
                    SELECT business_id, stars,
                    TRIM(A.value) AS category
                    FROM yelp_businesses,
                    LATERAL SPLIT_TO_TABLE(categories, ',') A
                )

                SELECT category, COUNT(DISTINCT business_id) AS num_of_businesses, ROUND(AVG(stars), 2) AS avg_stars
                FROM cte
                GROUP BY 1
                ORDER BY 2 DESC
                LIMIT 20;
                 ''').fetch_pandas_all()

display(df)

,CATEGORY,NUM_OF_BUSINESSES,AVG_STARS
0,Restaurants,52268,3.77
1,Food,27781,3.93
2,Shopping,24395,3.86
3,Home Services,14356,3.69
4,Beauty & Spas,14292,4.05
5,Nightlife,12281,3.91
6,Health & Medical,11890,3.86
7,Local Services,11198,3.85
8,Bars,11065,3.90
9,Automotive,10773,3.75


In [ ]:
# Bottom 20 categories of all businesses on Yelp with the lowest star rating.
df = cur.execute('''
                 WITH cte AS (
                    -- split the categories column so each business has a row for each category it has
                    SELECT business_id, stars,
                    TRIM(A.value) AS category
                    FROM yelp_businesses,
                    LATERAL SPLIT_TO_TABLE(categories, ',') A
                )

                SELECT category, COUNT(DISTINCT business_id) AS num_of_businesses, ROUND(AVG(stars), 2) AS avg_stars
                FROM cte
                GROUP BY 1  
                ORDER BY 2, ROUND(AVG(stars), 2)
                LIMIT 20;
                 ''').fetch_pandas_all()

display(df)


,CATEGORY,NUM_OF_BUSINESSES,AVG_STARS
0,Foot Care,1,2.00
1,Pathologists,1,2.00
2,DUI Schools,1,2.00
3,Trade Fairs,1,3.00
4,Pita,1,3.00
5,Town Hall,1,3.00
6,Fuel Docks,1,3.00
7,Osteopaths,1,3.00
8,Roman,1,3.00
9,Hospitalists,1,3.00


#### Review Volume by Category

In [ ]:
# Top 20 categories by number of reviews
df = cur.execute('''
                 WITH cte AS (
                    -- split the categories column so each business has a row for each category it has
                    SELECT business_id, stars, review_count,
                    TRIM(A.value) AS category
                    FROM yelp_businesses,
                    LATERAL SPLIT_TO_TABLE(categories, ',') A
                )

                SELECT category, SUM(review_count) AS num_of_reviews, COUNT(DISTINCT business_id) AS num_of_businesses, ROUND(AVG(stars), 2) AS avg_stars
                FROM cte
                GROUP BY 1  
                ORDER BY 2 DESC
                LIMIT 20;
                 ''').fetch_pandas_all()

display(df)

,CATEGORY,NUM_OF_REVIEWS,NUM_OF_BUSINESSES,AVG_STARS
0,Restaurants,4561279,52268,3.77
1,Food,1752281,27781,3.93
2,Nightlife,1488163,12281,3.91
3,Bars,1406415,11065,3.90
4,American (Traditional),976483,8139,3.65
5,American (New),950560,6097,3.84
6,Breakfast & Brunch,839467,6239,3.82
7,Sandwiches,669139,8366,3.80
8,Seafood,600183,3539,3.92
9,Event Planning & Services,591442,9895,3.93


#### Average Star Rating by Category

In [ ]:
# Top 20 categories by average star rating (minimum 1000 reviews)
df = cur.execute('''
                 WITH cte AS (
                    -- split the categories column so each business has a row for each category it has
                    SELECT business_id, stars, review_count,
                    TRIM(A.value) AS category
                    FROM yelp_businesses,
                    LATERAL SPLIT_TO_TABLE(categories, ',') A
                )

                SELECT category, SUM(review_count) AS num_of_reviews, COUNT(DISTINCT business_id) AS num_of_businesses, ROUND(AVG(stars), 2) AS avg_stars
                FROM cte
                GROUP BY 1  
                HAVING SUM(review_count) >= 1000
                ORDER BY 4 DESC
                LIMIT 20;
                 ''').fetch_pandas_all()

display(df)

,CATEGORY,NUM_OF_REVIEWS,NUM_OF_BUSINESSES,AVG_STARS
0,Conveyor Belt Sushi,1128,1,5.00
1,Art Tours,1962,18,4.94
2,Bike tours,1714,32,4.88
3,Sugaring,1549,39,4.85
4,Cideries,1650,25,4.84
5,Beer Tours,2949,51,4.82
6,Reiki,3048,201,4.82
7,Barre Classes,2926,158,4.80
8,Axe Throwing,1856,53,4.79
9,Meditation Centers,1451,99,4.79


In [16]:
# Bottom 20 categories by average star rating (minimum 1000 reviews)
df = cur.execute('''
                 WITH cte AS (
                    -- split the categories column so each business has a row for each category it has
                    SELECT business_id, stars, review_count,
                    TRIM(A.value) AS category
                    FROM yelp_businesses,
                    LATERAL SPLIT_TO_TABLE(categories, ',') A
                )

                SELECT category, SUM(review_count) AS num_of_reviews, COUNT(DISTINCT business_id) AS num_of_businesses, ROUND(AVG(stars), 2) AS avg_stars
                FROM cte
                GROUP BY 1  
                HAVING SUM(review_count) >= 1000
                ORDER BY 4
                LIMIT 20;
                 ''').fetch_pandas_all()

display(df)

,CATEGORY,NUM_OF_REVIEWS,NUM_OF_BUSINESSES,AVG_STARS
0,Television Service Providers,4253,159,2.33
1,University Housing,2630,122,2.53
2,Internet Service Providers,7947,356,2.60
3,Property Management,12282,759,2.74
4,Post Offices,5284,327,2.79
5,Apartments,28510,1921,2.82
6,Truck Rental,3633,206,2.84
7,Fast Food,224529,6472,2.89
8,Airlines,6018,118,2.89
9,Utilities,2276,102,2.90


## Reviews Analysis

#### Total Review Volume by Star Rating

In [45]:
# Count of reviews for each star value 1-5
df = cur.execute('''
                 SELECT '1 Star' AS review_rating, COUNT(review_id) AS num_reviews FROM yelp_reviews WHERE stars = 1
                 UNION
                 SELECT '2 Star' AS review_rating, COUNT(review_id) AS num_reviews FROM yelp_reviews WHERE stars = 2
                 UNION
                 SELECT '3 Star' AS review_rating, COUNT(review_id) AS num_reviews FROM yelp_reviews WHERE stars = 3
                 UNION
                 SELECT '4 Star' AS review_rating, COUNT(review_id) AS num_reviews FROM yelp_reviews WHERE stars = 4
                 UNION
                 SELECT '5 Star' AS review_rating, COUNT(review_id) AS num_reviews FROM yelp_reviews WHERE stars = 5
                 ORDER BY num_reviews DESC;
                 ''').fetch_pandas_all()

display(df)

,REVIEW_RATING,NUM_REVIEWS
0,5 Star,3231627
1,4 Star,1452918
2,1 Star,1069561
3,3 Star,691934
4,2 Star,544240


#### Volume of Each Rating by Business

In [53]:
# Count of reviews with each star value by business
df = cur.execute('''
                 SELECT 
                    business_id, 
                    SUM(CASE WHEN stars=1 THEN 1 ELSE 0 END) AS one_star_reviews,
                    SUM(CASE WHEN stars=2 THEN 1 ELSE 0 END) AS two_star_reviews,
                    SUM(CASE WHEN stars=3 THEN 1 ELSE 0 END) AS three_star_reviews,
                    SUM(CASE WHEN stars=4 THEN 1 ELSE 0 END) AS four_star_reviews,
                    SUM(CASE WHEN stars=5 THEN 1 ELSE 0 END) AS five_star_reviews,
                    COUNT(*) AS total_reviews,
                    SUM(stars) AS total_stars
                 FROM yelp_reviews
                 GROUP BY 1;
                 ''').fetch_pandas_all()

display(df)

,BUSINESS_ID,ONE_STAR_REVIEWS,TWO_STAR_REVIEWS,THREE_STAR_REVIEWS,FOUR_STAR_REVIEWS,FIVE_STAR_REVIEWS,TOTAL_REVIEWS,TOTAL_STARS
0,lCjNNCJKMEeAHjSUFqEH5A,0,0,0,3,6,9,42
1,nwN92Uje-xIKE5voPTTvBQ,18,28,36,102,425,609,2715
2,8OGLDw2Z0UK0aURh8eRVMQ,15,8,5,2,5,35,79
3,S8ZFYEgMejpChID8tzKo9A,79,115,236,658,1074,2162,9019
4,YRw-uBpdzRZngN6zzoC4WA,2,2,1,2,7,14,52
...,...,...,...,...,...,...,...,...
150341,I6p837glbhDog1XB4NSvKA,0,0,2,2,1,5,19
150342,5LIBxc9PNCGjtGk1I7HOpg,0,0,0,0,5,5,25
150343,615-GZzUg2Xpw-AkzOeXKA,3,0,0,0,2,5,13
150344,HEP7VxLY2v0mRDXPARrPyQ,1,0,0,0,4,5,21


In [62]:
# True star average for each business and percentage of five star and one star reviews to total reviews
df = cur.execute('''
                 WITH cte AS (
                    SELECT 
                        business_id, 
                        SUM(CASE WHEN stars=1 THEN 1 ELSE 0 END) AS one_star_reviews,
                        SUM(CASE WHEN stars=2 THEN 1 ELSE 0 END) AS two_star_reviews,
                        SUM(CASE WHEN stars=3 THEN 1 ELSE 0 END) AS three_star_reviews,
                        SUM(CASE WHEN stars=4 THEN 1 ELSE 0 END) AS four_star_reviews,
                        SUM(CASE WHEN stars=5 THEN 1 ELSE 0 END) AS five_star_reviews,
                        COUNT(*) AS total_reviews,
                        SUM(stars) AS total_stars
                    FROM yelp_reviews
                    GROUP BY 1
                 )

                 SELECT business_id, total_reviews, total_stars,
                    ROUND(total_stars / total_reviews, 2) AS true_avg_stars,
                    ROUND(five_star_reviews / total_reviews * 100, 2) AS five_star_pct,
                    ROUND(one_star_reviews / total_reviews * 100, 2) AS one_star_pct,
                 FROM cte
                 ORDER BY 5 DESC, 2 DESC
                 LIMIT 20;
                 ''').fetch_pandas_all()

display(df)

,BUSINESS_ID,TOTAL_REVIEWS,TOTAL_STARS,TRUE_AVG_STARS,FIVE_STAR_PCT,ONE_STAR_PCT
0,1RqfozJoosHAsKZhc5PY7w,114,570,5.00,100.00,0.00
1,-siOxQQcGKtb-04dX0Cxnw,104,520,5.00,100.00,0.00
2,4-P4Bzqd01YvKX9tp7IGfQ,90,450,5.00,100.00,0.00
3,dhLARBhUnJloLa8xZ1Stpw,78,390,5.00,100.00,0.00
4,LTqm4uY4GIYHfzuh5pVZNQ,77,385,5.00,100.00,0.00
5,x-SCuOwghy4GlZdVOKjt4g,74,370,5.00,100.00,0.00
6,2FQoAp9w0G_NhuZMqo9bfA,73,365,5.00,100.00,0.00
7,DnkpXhc5DKdeBT-0jG13JQ,70,350,5.00,100.00,0.00
8,NLx84Im37HcZkcFBO2cxlw,69,345,5.00,100.00,0.00
9,_LHkm-m9a_IDqfqfdWggHQ,68,340,5.00,100.00,0.00
